In [2]:
import cv2
import numpy as np
import tensorflow as tf
import threading
import pygame

pygame.mixer.init()
beep = pygame.mixer.Sound("267555__alienxxx__beep_sequence_02.wav") 

def play_beep():
    beep.play()

interpreter = tf.lite.Interpreter(model_path="owais_yolo_model_float32.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# === VIDEO SETUP ===
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
input_size = (800, 800)  # ⚠️ Match your model input size exactly
conf_threshold = 0.4
iou_threshold = 0.5
DROWNING_CLASS_INDEX = 0  # ✅ Set this based on your model’s class index

# === IOU / NMS ===
def compute_iou(box1, box2):
    x1, y1 = max(box1[0], box2[0]), max(box1[1], box2[1])
    x2, y2 = min(box1[2], box2[2]), min(box1[3], box2[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    union_area = (box1[2] - box1[0]) * (box1[3] - box1[1]) + \
                 (box2[2] - box2[0]) * (box2[3] - box2[1]) - inter_area
    return inter_area / union_area if union_area != 0 else 0

def nms(boxes, iou_thresh):
    boxes = sorted(boxes, key=lambda x: x[4], reverse=True)
    final = []
    while boxes:
        chosen = boxes.pop(0)
        final.append(chosen)
        boxes = [b for b in boxes if compute_iou(chosen, b) < iou_thresh]
    return final

# === MAIN LOOP ===
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("❌ Frame not captured")
        break

    orig_h, orig_w = frame.shape[:2]
    resized = cv2.resize(frame, input_size)
    input_tensor = np.expand_dims(resized, axis=0).astype(np.float32) / 255.0

    interpreter.set_tensor(input_details[0]['index'], input_tensor)
    interpreter.invoke()

    output = interpreter.get_tensor(output_details[0]['index'])[0]
    boxes = []
    drowning_detected = False

    for row in output:
        conf = row[4]
        if conf < conf_threshold:
            continue
        class_probs = row[5:]
        class_id = np.argmax(class_probs)
        score = class_probs[class_id]
        total_conf = conf * score

        if total_conf < conf_threshold:
            continue

        cx, cy, w, h = row[0], row[1], row[2], row[3]
        x1 = int((cx - w / 2) * orig_w / input_size[0])
        y1 = int((cy - h / 2) * orig_h / input_size[1])
        x2 = int((cx + w / 2) * orig_w / input_size[0])
        y2 = int((cy + h / 2) * orig_h / input_size[1])

        boxes.append([x1, y1, x2, y2, total_conf, class_id])

    filtered = nms(boxes, iou_threshold)

    for x1, y1, x2, y2, conf, class_id in filtered:
        color = (0, 0, 255) if class_id == DROWNING_CLASS_INDEX else (0, 255, 0)
        label = f"Class {class_id} {conf:.2f}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, label, (x1, max(20, y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        if class_id == DROWNING_CLASS_INDEX:
            drowning_detected = True

    if drowning_detected:
        threading.Thread(target=play_beep, daemon=True).start()

    cv2.imshow("Drowning Detection - TFLite", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# === CLEANUP ===
cap.release()
cv2.destroyAllWindows()
pygame.mixer.quit()


KeyboardInterrupt: 

HAVE SOME IMPRVEMENT

In [4]:
import cv2
import numpy as np
import tensorflow as tf
import threading
import pygame
import time

# Initialize pygame for sound
pygame.mixer.init()
beep = pygame.mixer.Sound("267555__alienxxx__beep_sequence_02.wav")

def play_beep():
    beep.play()

# Load TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path="owais_yolo_model_float32.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# === VIDEO SETUP ===
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)  # Reduce capture resolution
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
input_size = (800, 800)  # Reduced input size for faster processing
conf_threshold = 0.4
iou_threshold = 0.5
DROWNING_CLASS_INDEX = 0  # Set this based on your model’s class index

# === IOU / NMS ===
def compute_iou(box1, box2):
    x1, y1 = max(box1[0], box2[0]), max(box1[1], box2[1])
    x2, y2 = min(box1[2], box2[2]), min(box1[3], box2[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    union_area = (box1[2] - box1[0]) * (box1[3] - box1[1]) + \
                 (box2[2] - box2[0]) * (box2[3] - box2[1]) - inter_area
    return inter_area / union_area if union_area != 0 else 0

def nms(boxes, iou_thresh):
    boxes = sorted(boxes, key=lambda x: x[4], reverse=True)
    final = []
    while boxes:
        chosen = boxes.pop(0)
        final.append(chosen)
        boxes = [b for b in boxes if compute_iou(chosen, b) < iou_thresh]
    return final

# === MAIN LOOP ===
start_time = time.time()
frame_count = 0
beep_playing = False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("❌ Frame not captured")
        break

    frame_count += 1
    orig_h, orig_w = frame.shape[:2]
    resized = cv2.resize(frame, input_size)
    input_tensor = np.expand_dims(resized, axis=0).astype(np.float32) / 255.0

    interpreter.set_tensor(input_details[0]['index'], input_tensor)
    interpreter.invoke()

    output = interpreter.get_tensor(output_details[0]['index'])[0]
    boxes = []
    drowning_detected = False

    for row in output:
        conf = row[4]
        if conf < conf_threshold:
            continue
        class_probs = row[5:]
        class_id = np.argmax(class_probs)
        score = class_probs[class_id]
        total_conf = conf * score

        if total_conf < conf_threshold:
            continue

        cx, cy, w, h = row[0], row[1], row[2], row[3]
        x1 = int((cx - w / 2) * orig_w / input_size[0])
        y1 = int((cy - h / 2) * orig_h / input_size[1])
        x2 = int((cx + w / 2) * orig_w / input_size[0])
        y2 = int((cy + h / 2) * orig_h / input_size[1])

        boxes.append([x1, y1, x2, y2, total_conf, class_id])

    filtered = nms(boxes, iou_threshold)

    for x1, y1, x2, y2, conf, class_id in filtered:
        color = (0, 0, 255) if class_id == DROWNING_CLASS_INDEX else (0, 255, 0)
        label = f"Class {class_id} {conf:.2f}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, label, (x1, max(20, y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        if class_id == DROWNING_CLASS_INDEX:
            drowning_detected = True

    if drowning_detected and not beep_playing:
        beep_playing = True
        threading.Thread(target=play_beep, daemon=True).start()

    # Reset beep flag after a short delay
    if not drowning_detected:
        beep_playing = False

    # Display FPS every 30 frames
    if frame_count % 30 == 0:
        fps = frame_count / (time.time() - start_time)
        print(f"FPS: {fps:.2f}")

    cv2.imshow("Drowning Detection - TFLite", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# === CLEANUP ===
cap.release()
cv2.destroyAllWindows()
pygame.mixer.quit()

FPS: 0.99
FPS: 1.00


KeyboardInterrupt: 

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path="owais_yolo_model_float32.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Set parameters
input_size = (800, 800)  # Adjust to match your model's input size
conf_threshold = 0.4
DROWNING_CLASS_INDEX = 0  # Set this based on your model's class index

# Function to preprocess the image
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    orig_h, orig_w = image.shape[:2]
    resized = cv2.resize(image, input_size)
    input_tensor = np.expand_dims(resized, axis=0).astype(np.float32) / 255.0
    return image, input_tensor, orig_h, orig_w

# Function to postprocess the output
def postprocess_output(output, orig_h, orig_w):
    boxes = []
    for row in output:
        conf = row[4]
        if conf < conf_threshold:
            continue
        class_probs = row[5:]
        class_id = np.argmax(class_probs)
        score = class_probs[class_id]
        total_conf = conf * score

        if total_conf < conf_threshold:
            continue

        cx, cy, w, h = row[0], row[1], row[2], row[3]
        x1 = int((cx - w / 2) * orig_w / input_size[0])
        y1 = int((cy - h / 2) * orig_h / input_size[1])
        x2 = int((cx + w / 2) * orig_w / input_size[0])
        y2 = int((cy + h / 2) * orig_h / input_size[1])

        boxes.append((x1, y1, x2, y2, total_conf, class_id))
    return boxes

# Load and preprocess the image
image_path = "IMG_20250410_233137696.jpg"  # Replace with your image path
image, input_tensor, orig_h, orig_w = preprocess_image(image_path)

# Run inference
interpreter.set_tensor(input_details[0]['index'], input_tensor)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])[0]

# Postprocess the output
boxes = postprocess_output(output, orig_h, orig_w)

# Draw the detections on the image
for x1, y1, x2, y2, conf, class_id in boxes:
    color = (0, 0, 255) if class_id == DROWNING_CLASS_INDEX else (0, 255, 0)
    label = f"Class {class_id} {conf:.2f}"
    cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
    cv2.putText(image, label, (x1, max(20, y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

# Display the image
cv2.imshow("Detection Result", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

import cv2
import numpy as np
import tensorflow as tf

# Load TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path="owais_yolo_model_float32.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Set parameters
input_size = (416, 416)  # Reduced input size for faster processing
conf_threshold = 0.4
DROWNING_CLASS_INDEX = 0  # Set this based on your model's class index

# Function to preprocess the image
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    orig_h, orig_w = image.shape[:2]
    resized = cv2.resize(image, input_size)
    input_tensor = np.expand_dims(resized, axis=0).astype(np.float32) / 255.0
    return image, input_tensor, orig_h, orig_w

# Function to postprocess the output
def postprocess_output(output, orig_h, orig_w):
    boxes = []
    for row in output:
        conf = row[4]
        if conf < conf_threshold:
            continue
        class_probs = row[5:]
        class_id = np.argmax(class_probs)
        score = class_probs[class_id]
        total_conf = conf * score

        if total_conf < conf_threshold:
            continue

        cx, cy, w, h = row[0], row[1], row[2], row[3]
        x1 = int((cx - w / 2) * orig_w / input_size[0])
        y1 = int((cy - h / 2) * orig_h / input_size[1])
        x2 = int((cx + w / 2) * orig_w / input_size[0])
        y2 = int((cy + h / 2) * orig_h / input_size[1])

        boxes.append((x1, y1, x2, y2, total_conf, class_id))
    return boxes

# Load and preprocess the image
image_path = "IMG_20250410_233137696.jpg"  # Replace with your image path
image, input_tensor, orig_h, orig_w = preprocess_image(image_path)

# Run inference
interpreter.set_tensor(input_details[0]['index'], input_tensor)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])[0]

# Postprocess the output
boxes = postprocess_output(output, orig_h, orig_w)

# Draw the detections on the image
for x1, y1, x2, y2, conf, class_id in boxes:
    color = (0, 0, 255) if class_id == DROWNING_CLASS_INDEX else (0, 255, 0)
    label = f"Class {class_id} {conf:.2f}"
    cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
    cv2.putText(image, label, (x1, max(20, y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

# Display the image
cv2.imshow("Detection Result", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
import cv2
import numpy as np
import tensorflow as tf

# Load TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path="./owais_yolo_model_float16.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Set parameters
input_size = (800, 800)  # Reduced input size for faster processing
conf_threshold = 0.4
DROWNING_CLASS_INDEX = 0  # Set this based on your model's class index

# Function to preprocess the image
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    orig_h, orig_w = image.shape[:2]
    resized = cv2.resize(image, input_size)
    input_tensor = np.expand_dims(resized, axis=0).astype(np.float32) / 255.0
    return image, input_tensor, orig_h, orig_w

# Function to postprocess the output
def postprocess_output(output, orig_h, orig_w):
    boxes = []
    for row in output:
        conf = row[4]
        if conf < conf_threshold:
            continue
        class_probs = row[5:]
        class_id = np.argmax(class_probs)
        score = class_probs[class_id]
        total_conf = conf * score

        if total_conf < conf_threshold:
            continue

        cx, cy, w, h = row[0], row[1], row[2], row[3]
        x1 = int((cx - w / 2) * orig_w / input_size[0])
        y1 = int((cy - h / 2) * orig_h / input_size[1])
        x2 = int((cx + w / 2) * orig_w / input_size[0])
        y2 = int((cy + h / 2) * orig_h / input_size[1])

        boxes.append((x1, y1, x2, y2, total_conf, class_id))
    return boxes

# Load and preprocess the image
image_path = "IMG_20250410_233137696.jpg"  # Replace with your image path
image, input_tensor, orig_h, orig_w = preprocess_image(image_path)

# Run inference
interpreter.set_tensor(input_details[0]['index'], input_tensor)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])[0]

# Postprocess the output
boxes = postprocess_output(output, orig_h, orig_w)

# Draw the detections on the image
for x1, y1, x2, y2, conf, class_id in boxes:
    color = (0, 0, 255) if class_id == DROWNING_CLASS_INDEX else (0, 255, 0)
    label = f"Class {class_id} {conf:.2f}"
    cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
    cv2.putText(image, label, (x1, max(20, y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

# Display the image
cv2.imshow("Detection Result", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'